In [14]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [6]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Creating Graph Function

In [7]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Extracting Stock Data

In [8]:
appl = yf.Ticker("AAPL")
tesla_data = appl.history(period="max")
tesla_data.reset_index(inplace = True)

## Webscraping Tesla Revenue

In [9]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text

In [10]:
soup = BeautifulSoup(html_data, 'html.parser')

In [29]:
table = soup.find_all("tbody")[1]

dates = []
revenues = []

for row in table.find_all("tr"):
    cols = row.find_all("td")
    date = cols[0].text.strip()
    revenue = cols[1].text.strip()
    dates.append(date)
    revenues.append(revenue)
    
# Create DataFrame
tesla_revenue = pd.DataFrame({
    "Date": dates,
    "Revenue": revenues
})

# Clean revenue values
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(',', '').str.replace('$', '')
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors='coerce')

tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'], errors='coerce')
print("Data type of 'Date' column after conversion:", tesla_revenue['Date'].dtype)

# Convert revenue values to float
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].astype(float)

Data type of 'Date' column after conversion: datetime64[ns]


In [30]:
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].astype(str).str.replace(',', '').str.replace('$', '')

In [31]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [32]:
tesla_revenue.tail

<bound method NDFrame.tail of          Date  Revenue
0  2022-09-30  21454.0
1  2022-06-30  16934.0
2  2022-03-31  18756.0
3  2021-12-31  17719.0
4  2021-09-30  13757.0
5  2021-06-30  11958.0
6  2021-03-31  10389.0
7  2020-12-31  10744.0
8  2020-09-30   8771.0
9  2020-06-30   6036.0
10 2020-03-31   5985.0
11 2019-12-31   7384.0
12 2019-09-30   6303.0
13 2019-06-30   6350.0
14 2019-03-31   4541.0
15 2018-12-31   7226.0
16 2018-09-30   6824.0
17 2018-06-30   4002.0
18 2018-03-31   3409.0
19 2017-12-31   3288.0
20 2017-09-30   2985.0
21 2017-06-30   2790.0
22 2017-03-31   2696.0
23 2016-12-31   2285.0
24 2016-09-30   2298.0
25 2016-06-30   1270.0
26 2016-03-31   1147.0
27 2015-12-31   1214.0
28 2015-09-30    937.0
29 2015-06-30    955.0
30 2015-03-31    940.0
31 2014-12-31    957.0
32 2014-09-30    852.0
33 2014-06-30    769.0
34 2014-03-31    621.0
35 2013-12-31    615.0
36 2013-09-30    431.0
37 2013-06-30    405.0
38 2013-03-31    562.0
39 2012-12-31    306.0
40 2012-09-30     50.0
41 2

## Extracting Gamestop Data

In [35]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)

## Webscraping Gamestop Revenue

In [36]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

response = requests.get(url)
html_data = response.text

In [37]:
soup = BeautifulSoup(html_data, 'html.parser')

In [38]:
table = soup.find_all("tbody")[1]

dates = []
revenues = []

for row in table.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace(',', '').replace('$', '')
        dates.append(date)
        revenues.append(revenue)

gme_revenue = pd.DataFrame({
    "Date": dates,
    "Revenue": revenues
})

print(gme_revenue.head())

         Date Revenue
0  2020-04-30    1021
1  2020-01-31    2194
2  2019-10-31    1439
3  2019-07-31    1286
4  2019-04-30    1548


In [39]:
gme_revenue.tail()

Date Revenue
57  2006-01-31    1667
58  2005-10-31     534
59  2005-07-31     416
60  2005-04-30     475
61  2005-01-31     709

## Telsa Graph

In [40]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

C:\Users\Houst\AppData\Local\Temp\ipykernel_56212\2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\Houst\AppData\Local\Temp\ipykernel_56212\2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



## Gamestop Graph

In [41]:
make_graph(gme_data, gme_revenue, 'GameStop')

C:\Users\Houst\AppData\Local\Temp\ipykernel_56212\2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\Houst\AppData\Local\Temp\ipykernel_56212\2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

